In [1]:
import psycopg2
import pandas as pd
import numpy as np
import mariadb
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [2]:
water_df = pd.read_csv("input/water_analysis_cleaned.csv")

In [3]:
water_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.270,NaN
1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.250,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",ph,7.250,NaN
3,3,3,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",electrical_conductivity,0.150,mS cm -1
4,4,4,CB012WA0052,2012-07-19 00:00:00.000,"Irrigation Water Analysis,Feacal Coliforms (MP...",ammonium,0.010,ppm
...,...,...,...,...,...,...,...,...
1240310,1240318,1240318,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),nitrate_n,5.900,ppm
1240311,1240319,1240319,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),phosphorus,0.012,ppm
1240312,1240320,1240320,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),sulphur,5.270,ppm
1240313,1240321,1240321,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),iron,0.013,ppm


In [4]:
water_df['analysis_name'] = [ str(i).split(",") for i in water_df['analysis_name'] ]

In [5]:
water_df.head()

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CA335WA0010,2019-04-18 00:00:00.000,[Reverse Osmosis Water Use],ph,8.27,NaN
1,1,1,CD026WA2057,2019-05-07 00:00:00.000,[Reverse Osmosis Water Use],ph,8.25,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,"[Irrigation Water Analysis, Feacal Coliforms (...",ph,7.25,NaN
3,3,3,CB012WA0052,2012-07-19 00:00:00.000,"[Irrigation Water Analysis, Feacal Coliforms (...",electrical_conductivity,0.15,mS cm -1
4,4,4,CB012WA0052,2012-07-19 00:00:00.000,"[Irrigation Water Analysis, Feacal Coliforms (...",ammonium,0.01,ppm


In [6]:
_df = water_df.explode("analysis_name")
_df.to_csv("output/water_single_analysis.csv")

In [7]:
_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.270,NaN
1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.250,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,ph,7.250,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ph,7.250,NaN
3,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,electrical_conductivity,0.150,mS cm -1
...,...,...,...,...,...,...,...,...
1240310,1240318,1240318,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),nitrate_n,5.900,ppm
1240311,1240319,1240319,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),phosphorus,0.012,ppm
1240312,1240320,1240320,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),sulphur,5.270,ppm
1240313,1240321,1240321,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),iron,0.013,ppm


In [8]:
_df = pd.read_csv("output/water_single_analysis.csv")

In [9]:
_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.270,NaN
1,1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.250,NaN
2,2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,ph,7.250,NaN
3,2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ph,7.250,NaN
4,3,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,electrical_conductivity,0.150,mS cm -1
...,...,...,...,...,...,...,...,...,...
1749188,1240310,1240318,1240318,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),nitrate_n,5.900,ppm
1749189,1240311,1240319,1240319,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),phosphorus,0.012,ppm
1749190,1240312,1240320,1240320,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),sulphur,5.270,ppm
1749191,1240313,1240321,1240321,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),iron,0.013,ppm


In [10]:
analysisDetails = pd.read_csv("output/analysis_chemicals.csv")

In [11]:
analysisDetails.analysis_name = [ i.replace(":","") for i in analysisDetails.analysis_name ]

In [10]:
_df.to_csv("output/water_single_analysis.csv",chunksize=1000)

In [11]:
water_df = _df.copy(deep=True)

In [12]:
np.unique([ i for i in water_df.analysis_name if "Reverse" in i])

array(['Reverse Osmosis Water Use'], dtype='<U25')

In [13]:
np.unique([ i for i in water_df.analysis_name if "Dist" in i])

array([], dtype=float64)

In [14]:
np.unique([ i for i in water_df.analysis_name if "(FAO)" in i])

array(['Chemical & Heavy Metals Water Analysis (FAO) ',
       'Irrigation Water Analysis (FAO)'], dtype='<U45')

In [15]:
np.unique([ i for i in water_df.analysis_name if "Fruit" in i])

array([], dtype=float64)

In [16]:
np.unique([ i for i in water_df.analysis_name if "Reagent" in i])

array(['Reagent Water Analysis'], dtype='<U22')

In [17]:
np.unique([ i for i in water_df.analysis_name if "Tea" in i])

array(['Tea Extract Analysis'], dtype='<U20')

In [18]:
irrigation_analysis_names = np.unique([ i for i in water_df.analysis_name if "Irrigation" in i])

In [23]:
irrigation_analysis_names

array([' Irrigation Water Analysis', 'Basic Irrigation Water Analysis',
       'Irrigation Water Analysis', 'Irrigation Water Analysis (FAO)',
       'Standard Irrigation Water Analysis'], dtype='<U34')

In [19]:
analysis_freq = water_df.analysis_name.value_counts()

In [20]:
analysis_freq = analysis_freq.loc[analysis_freq.values >= 820]

In [21]:
# water_df = water_df.loc[water_df.analysis_name.isin(analysis_freq.index)]

In [22]:
water_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.270,NaN
1,1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.250,NaN
2,2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,ph,7.250,NaN
3,2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ph,7.250,NaN
4,3,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,electrical_conductivity,0.150,mS cm -1
...,...,...,...,...,...,...,...,...,...
1749188,1240310,1240318,1240318,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),nitrate_n,5.900,ppm
1749189,1240311,1240319,1240319,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),phosphorus,0.012,ppm
1749190,1240312,1240320,1240320,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),sulphur,5.270,ppm
1749191,1240313,1240321,1240321,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),iron,0.013,ppm


In [24]:
water_df.loc[water_df.analysis_name.isin(irrigation_analysis_names), "analysis_name"] = "Irrigation"

In [25]:
water_df = water_df.loc[water_df.analysis_name != 'Tea Extract Analysis']

In [22]:
water_df = water_df.loc[~(water_df['result'] == "ND") ]
water_df = water_df.loc[~(water_df['result'] == "Detected") ]

In [26]:
water_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.270,NaN
1,1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.250,NaN
2,2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation,ph,7.250,NaN
3,2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ph,7.250,NaN
4,3,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation,electrical_conductivity,0.150,mS cm -1
...,...,...,...,...,...,...,...,...,...
1749188,1240310,1240318,1240318,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),nitrate_n,5.900,ppm
1749189,1240311,1240319,1240319,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),phosphorus,0.012,ppm
1749190,1240312,1240320,1240320,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),sulphur,5.270,ppm
1749191,1240313,1240321,1240321,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),iron,0.013,ppm


In [24]:
# water_df = water_df.dropna(subset="result")

In [25]:
# water_df['result'] = [ float(i.replace("<","").replace(">","").strip()) for i in water_df['result']]

In [12]:
water_df['unit_name'] = [ str(i).strip() for i in water_df['unit_name'].values ]

In [13]:
# water_df['chemical_name'] = [ str(i).lower().replace(" ","_").replace("(","").replace(")","").replace(".","").strip() for i in water_df['chemical_name'].values ]

In [14]:
# water_df = water_df.loc[~(water_df['result'] < 0)]

In [15]:
df_ = water_df[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/water_unit_per_chemical.csv")

In [16]:
df_ = df_.reset_index()

In [17]:
water_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.270,nan
1,1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.250,nan
2,2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,ph,7.250,nan
3,2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ph,7.250,nan
4,3,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,electrical_conductivity,0.150,mS cm -1
...,...,...,...,...,...,...,...,...,...
1749188,1240310,1240318,1240318,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),nitrate_n,5.900,ppm
1749189,1240311,1240319,1240319,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),phosphorus,0.012,ppm
1749190,1240312,1240320,1240320,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),sulphur,5.270,ppm
1749191,1240313,1240321,1240321,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),iron,0.013,ppm


In [18]:
analysis_chem_unit_df = pd.DataFrame()
for analysis in df_['analysis_name'].unique():
    print(analysis)
    chems_count_ = df_.loc[df_['analysis_name'] == analysis]
    duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
    temp_analysis_chem_unit_df = pd.DataFrame()
    for index,row in duplicates_chem.iterrows():
        chemical = row['chemical_name']
        chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
        unit = chems_count_2.loc[chems_count_2['count'] == max(chems_count_2['count'])]['unit_name'].values[0]
        _ = pd.DataFrame({"analysis_name":[analysis],"chemical_name":[chemical], "unit_name":[unit]})
        temp_analysis_chem_unit_df = pd.concat([temp_analysis_chem_unit_df, _])
    analysis_chem_unit_df = pd.concat([analysis_chem_unit_df, temp_analysis_chem_unit_df])
analysis_chem_unit_df.to_csv("output/water_unit_per_chemical_decision.csv")

Irrigation Water Analysis
Standard Drinking Water Analysis (WHO)
Irrigation Water Analysis (FAO)
Standard Drinking Water Analysis
Basic Drinking Water Analysis
Feacal Coliforms (MPN) and E.Coli
Reverse Osmosis Water Use
Basic Drinking Water Analysis (WHO)
Heavy Metals in Water
Total Coliform and Faecal E.Coli
Base Titanium Water Analysis
E coli & Total Coliforms
Post Harvest Water Analysis
Standard Drinking Water
Total Suspended Solids
Total Bacteria Count
Total Dissolved Solids
KEBS Drinking Water Standard Analysis
KEBS Drinking Water Microbiology
Free Chlorine
Standard Drinking Water + Free Chlorine Analysis
Turbidity Analysis
Total Coliforms and Faecal E.Coli
Water Colour Analysis
Heavy Metal Analysis (Water)
Total Chlorine
Taita Water Analysis (Uni of Helsinki)
WRMA Water Analysis
Water Borne Diseases
Arsenic in Water
Total Bacterial Count
Aquasearch Full Chemical Analysis
Legionella
Alkalinity in Water
pH in Water
E coli
KEBS Drinking Water Minimum Monitoring
Chemical & Biochemica

In [19]:
analysis_chem_unit_df

,analysis_name,chemical_name,unit_name
0,Irrigation Water Analysis,magnesium,ppm
0,Irrigation Water Analysis,manganese,ppm
0,Irrigation Water Analysis,boron,ppm
0,Irrigation Water Analysis,sulphur,ppm
0,Irrigation Water Analysis,silicon__,ppm
...,...,...,...
0,KS Packaged Water Microbiology,salmonella_ssp,cfu/100 ml
0,KS Packaged Water Microbiology,staphylococcus_aureus,cfu/100 ml
0,Salmonella and E.Coli (Water),e_coli,cfu/100 ml
0,TVC @22,tvc_at_30_degrees,cfu/ml


In [20]:
df = pd.DataFrame()
for index, row in analysis_chem_unit_df.iterrows():
    analysis = row['analysis_name']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = water_df.loc[(water_df['analysis_name'] == analysis) & (water_df['chemical_name'] == chemical) & (water_df['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [21]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
10,6,6,6,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,magnesium,1.77,ppm
32,17,17,17,CB012WA0053,2012-07-19 00:00:00.000,Irrigation Water Analysis,magnesium,1.96,ppm
58,39,39,39,CB033WA0026,2014-12-08 00:00:00.000,Irrigation Water Analysis,magnesium,3.33,ppm
260,213,213,213,CO006WA0079,2012-07-19 00:00:00.000,Irrigation Water Analysis,magnesium,0.48,ppm
346,299,299,299,CP052WA0004,2012-07-13 00:00:00.000,Irrigation Water Analysis,magnesium,2.52,ppm
...,...,...,...,...,...,...,...,...,...
956345,650384,650384,650384,CD048WA0058,2018-11-19 00:00:00.000,TVC @22,tvc_at_30_degrees,690.00,cfu/ml
956379,650395,650395,650395,CD048WA0057,2018-11-19 00:00:00.000,TVC @22,tvc_at_30_degrees,660.00,cfu/ml
505162,340402,340402,340402,FA324-55WA0002,2016-03-14 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,feacal_e_coli,28.00,cfu/100 ml
515843,347816,347816,347816,FA324-55WA0003,2016-03-21 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,feacal_e_coli,0.00,cfu/100 ml


In [22]:
df.to_csv("output/water_analysis_cleaned.csv",chunksize=1000)

In [23]:
df[["analysis_name", "chemical_name", "unit_name"]].set_index('analysis_name').to_csv('output/expected_units/analysis_units.csv')

In [40]:
# df[["analysis_name", "chemical_name", "unit_name"]].set_index('analysis_name').to_dict('dict') 

In [40]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [41]:
water_df = water_df[['sample_code','analysis_name','chemical_name', 'result']]

In [42]:
water_df['analysis_name'] = [ i.split(",") for i in water_df['analysis_name'] ]

In [43]:
water_df['analysis_name']

0                                   [Irrigation]
1                                   [Irrigation]
2                                   [Irrigation]
3                                   [Irrigation]
4                                   [Irrigation]
                           ...                  
1006750    [KS Drinking Water Standard Analysis]
1006751    [KS Drinking Water Standard Analysis]
1006752    [KS Drinking Water Standard Analysis]
1006753    [KS Drinking Water Standard Analysis]
1006754    [KS Drinking Water Standard Analysis]
Name: analysis_name, Length: 1006755, dtype: object

In [45]:
water_df.analysis_name = [ i[0] for i in water_df.analysis_name ] 

In [48]:
water_df.chemical_name.value_counts()

bicarbonate               48654
ammonium                  48458
zinc                      46425
copper                    46419
boron                     46418
                          ...  
dissolved_silica             24
total_viable_count_@37       17
sodium_chloride               4
escherichia_coli              2
moisture                      2
Name: chemical_name, Length: 70, dtype: int64

In [46]:
df.to_csv("output/water_analysis_cleaned.csv",chunksize=1000)

In [ ]:
# _df = pd.DataFrame()
# for index, row in water_df.iterrows():
#     analyses = row['analysis_name']
#     sample_code = row['sample_code']
#     for analysis in analyses:
#         _ = pd.DataFrame(row).T
#         _['analysis_name'] = analysis
#         _df = pd.concat([_df, _])

# _df.to_csv("output/water_single_analysis.csv")

In [36]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [37]:
water_df_pivoted = pd.pivot_table(data=water_df, index="sample_code", columns="chemical_name", values="result")

In [39]:
water_df_pivoted.columns

Index(['aluminium', 'ammonium', 'arsenic', 'barium', 'bicarbonate', 'bod',
       'boron', 'cadmium', 'calcium', 'carbonate', 'charge_balance',
       'chloride', 'chlorides', 'chromium', 'cobalt', 'cod',
       'colloidal_silica', 'colour', 'copper', 'dissolved_silica', 'e_coli',
       'electrical_conductivity', 'escherichia_coli', 'faecal_e_coli',
       'feacal_coliforms', 'feacal_coliforms_mpn', 'feacal_e_coli',
       'fluorides', 'hardness', 'iodine', 'iron', 'lead', 'magnesium',
       'manganese', 'mercury', 'moisture', 'molybdenum', 'nickel', 'nitrate_n',
       'nitrates', 'nitrite', 'oil_&_grease', 'ph', 'phosphate', 'phosphorus',
       'potassium', 'pseudomonas', 'salmonella', 'sar', 'selenium', 'shigella',
       'silica', 'silicon__', 'silver', 'sodium', 'sodium_chloride',
       'strontium', 'sulphate', 'sulphates', 'sulphur', 'tbc_at_22_degrees',
       'tbc_at_37_degrees', 'total_coliforms', 'total_dissolved_solids',
       'total_suspended_solids', 'total_viable_cou

In [43]:
water_df_pivoted.total_suspended_solids.info()

<class 'pandas.core.series.Series'>
Index: 20522 entries, AGD102-124WA0001 to I48-1-9WA0001
Series name: total_suspended_solids
Non-Null Count  Dtype  
--------------  -----  
2564 non-null   float64
dtypes: float64(1)
memory usage: 836.7+ KB
